In [1]:
# Extended Kalman Implementation

In [2]:
def getVelocities(left_motor_speed, right_motor_speed): # From thymio angular motor.left.speed() and motor.right.speed(), get forward speed v and rotational speed omega: 
    wheel_radius = 5 
    wheelbase_radius = 10
    motortarget_to_speed = 20 # NEED TO REPLACE THESE CONSTANTS
    
    left_speed = motortarget_to_speed * left_motor_speed
    right_speed = motortarget_to_speed * right_motor_speed
    
    v = (left_speed + right_speed)*wheel_radius/2
    omega = (-left_speed + right_speed)*wheel_radius/wheelbase_radius
    
    return v, omega

In [3]:
getVelocities(100,100)

(10000.0, 0.0)

In [118]:
import numpy as np

# Example of instance: KF=KalmanFilter(0.1, [0, 0, 0, 0, 0]) (Ts is time between each measurement, x_ini is initial conditions)

# For update() function : need measurement vector z
# z[0:2] are the measurements from the camera (px, py, orientation)
# z[3:4] are the measurements from the Thymio (need to convert motor.left.speed() and motor.right.speed() to v and omega with the getVelocities() function.


class KalmanFilter():
    def __init__(self, Ts, x_ini):
        self.Ts = Ts

        # Initial State Vector
        self.x=np.matrix([[x_ini[0]], [x_ini[1]], [x_ini[2]], [x_ini[3]], [x_ini[4]]])
    

        # State Transition Matrix (Warning : changes with time -> Extended Kalman Filter)
        self.Fk=np.matrix([[1, 0, 0, 0, 0],
                           [0, 1, 0, 0, 0],
                           [0, 0, 1, 0, 0],
                           [0, 0, 0, 1, 0],
                           [0, 0, 0, 0, 1]])

        # Observation matrix (here all states px, py, theta, v, omega can be observed)
        self.H=np.matrix([[1, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0],
                          [0, 0, 1, 0, 0],
                          [0, 0, 0, 1, 0],
                          [0, 0, 0, 0, 1]])

        # Noise on system
        self.Q=np.matrix([[0.01, 0, 0, 0, 0],
                          [0, 0.01, 0, 0, 0],
                          [0, 0, 0.01, 0, 0],
                          [0, 0, 0, 0.01, 0],
                          [0, 0, 0, 0, 0.01]]) 

        # Noise on measurements
        self.R=np.matrix([[0.001, 0, 0, 0, 0],
                          [0, 0.001, 0, 0, 0],
                          [0, 0, 0.001, 0, 0],
                          [0, 0, 0, 0.5, 0],
                          [0, 0, 0, 0, 0.5]])

        self.P=np.matrix([[1, 0, 0, 0, 0],
                           [0, 1, 0, 0, 0],
                           [0, 0, 1, 0, 0],
                           [0, 0, 0, 1, 0],
                           [0, 0, 0, 0, 1]])
        
    def updateFk(self):
        self.Fk=np.matrix([[1, 0, -self.Ts*self.x[3]*np.sin(self.x[2]), self.Ts*np.cos(self.x[2]), 0],
                      [0, 1,  self.Ts*self.x[3]*np.cos(self.x[2]), self.Ts*np.sin(self.x[2]), 0],
                      [0, 0, 1, 0, self.Ts],
                      [0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 1]])
        self.Fk = self.Fk.astype(float)

    def predict(self):
        # Update Fk
        self.updateFk()
        # Predicted State Estimate (A priori)
        self.x=np.dot(self.Fk, self.x)
        # Predicted Covariance Estimate
        self.P=np.dot(np.dot(self.Fk, self.P), self.Fk.T)+self.Q
        return self.x

    def update(self, z, CameraAccessible = True): # z[0:2] corrpesonds to measurement of camera, z[3:4] corresponds to measurements of wheels
        
        if CameraAccessible:
            self.H=np.matrix([[1, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0],
                          [0, 0, 1, 0, 0],
                          [0, 0, 0, 1, 0],
                          [0, 0, 0, 0, 1]])
            
            self.R=np.matrix([[0.001, 0, 0, 0, 0],
                          [0, 0.001, 0, 0, 0],
                          [0, 0, 0.001, 0, 0],
                          [0, 0, 0, 0.5, 0],
                          [0, 0, 0, 0, 0.5]])
        else: # ------------------------------------------ WARNING : IF CAMERAACCESSIBLE = FALSE, Z IS A 2x1 VECTOR
            self.H=np.matrix([[0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 1]])
            
            self.R=np.matrix([[0.5, 0],
                              [0,0.5]])
            
            
        # Compute Kalman gain
        S=np.dot(self.H, np.dot(self.P, self.H.T))+self.R
        inv_S = np.linalg.pinv(S.astype(float))
        K=np.dot(np.dot(self.P, self.H.T),inv_S)

        # Correction / innovation
        self.x=self.x+np.dot(K, (z-np.dot(self.H, self.x)))
        self.x=np.round(self.x.astype(float))
        I=np.eye(self.H.shape[1])
        self.P=(I-(K*self.H))*self.P


        return self.x

In [119]:
KF=KalmanFilter(0.1, [0, 0, 0, 0, 0])

In [120]:
x_apriori = KF.predict().astype(np.int32)

In [121]:
# Test if code works for random measurement z (z[0:2] is camera measurements px, py, theta and z[3:4] is wheel measurements)

z = np.array([1,2,np.pi/3,1, 1])
z = np.expand_dims(z, axis=-1)

In [122]:
KF.predict()


matrix([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [127]:
z = np.array([2,2,np.pi/4,0,0])
z = np.expand_dims(z, axis=-1)

In [128]:
KF.predict()

matrix([[2.],
        [2.],
        [1.],
        [0.],
        [0.]])

In [129]:
KF.update(z)

matrix([[ 2.],
        [ 2.],
        [ 1.],
        [ 0.],
        [-0.]])

In [166]:
z = np.array([10, 0])
z = np.expand_dims(z, axis=-1)

In [167]:
KF.predict()
KF.update(z,False)

matrix([[-7.],
        [11.],
        [ 2.],
        [ 7.],
        [ 3.]])

In [141]:
z = np.array([2,2,np.pi/4,0,0])
z = np.expand_dims(z, axis=-1)

In [86]:
KF.predict()
KF.update(z)

matrix([[ 2.],
        [ 2.],
        [ 1.],
        [ 4.],
        [-0.]])

In [87]:
KF.predict()
KF.update(z)

matrix([[ 2.],
        [ 2.],
        [ 1.],
        [ 3.],
        [-0.]])

In [88]:
KF.predict()
KF.update(z)

matrix([[ 2.],
        [ 2.],
        [ 1.],
        [ 2.],
        [-0.]])

In [89]:
z = np.array([100,100,np.pi/4,50,0])
z = np.expand_dims(z, axis=-1)

In [90]:
KF.predict()

matrix([[1.93976626],
        [2.27635466],
        [1.        ],
        [2.        ],
        [0.        ]])

In [91]:
KF.update(z)

matrix([[ 92.],
        [ 93.],
        [  1.],
        [100.],
        [ -0.]])

In [92]:
KF.predict()

matrix([[ 88.98831321],
        [106.81773291],
        [  1.        ],
        [100.        ],
        [  0.        ]])

In [93]:
KF.update(z)

matrix([[100.],
        [100.],
        [  1.],
        [ 93.],
        [ -0.]])

In [ ]:
Ts = 0.1  # time tick [s]
SIM_TIME = 50.0  # simulation time [s]

show_animation = True